In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Install dependencies.
!pip install bitsandbytes transformers accelerate peft -q

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git -q

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora_sdxl.py

In [ ]:
from huggingface_hub import snapshot_download

# Set the local directory where the dataset will be saved
local_dir = "./3d_icon_dataset/"

# Download the dataset from Hugging Face
snapshot_download(
    repo_id="linoyts/3d_icon",
    local_dir=local_dir,
    repo_type="dataset",
    ignore_patterns=[".gitattributes"]
)


In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols, resize=256):

    if resize is not None:
        imgs = [img.resize((resize, resize)) for img in imgs]
    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid

In [ ]:
import glob

# change path to display images from your local dir
img_paths = "./3d_icon_dataset/*.jpg"
imgs = [Image.open(path) for path in glob.glob(img_paths)]

num_imgs_to_preview = 5
image_grid(imgs[:num_imgs_to_preview], 1, num_imgs_to_preview)

In [ ]:
import requests
from transformers import AutoProcessor, BlipForConditionalGeneration
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# load the processor and the captioning model
blip_processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base",torch_dtype=torch.float16).to(device)

# captioning utility
def caption_images(input_image):
    inputs = blip_processor(images=input_image, return_tensors="pt").to(device, torch.float16)
    pixel_values = inputs.pixel_values

    generated_ids = blip_model.generate(pixel_values=pixel_values, max_length=50)
    generated_caption = blip_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_caption

In [ ]:
import glob
from PIL import Image

# create a list of (Pil.Image, path) pairs
local_dir = "./3d_icon_dataset/"
imgs_and_paths = [(path,Image.open(path)) for path in glob.glob(f"{local_dir}*.jpg")]

In [ ]:
import matplotlib.pyplot as plt

# Number of images to show
num_images_to_show = 5

# Display images and their generated captions
for path, img in imgs_and_paths[:num_images_to_show]:
    # Generate caption
    caption = caption_images(img)

    # Display image and caption
    plt.figure(figsize=(5, 5))
    plt.imshow(img)
    plt.axis("off")
    plt.title(caption, fontsize=12)
    print(f"🖼 {path}\n📝 Caption: {caption}\n")
    plt.show()


In [ ]:
import json

caption_prefix = "In the style of TOK, " 
with open(f'{local_dir}metadata.jsonl', 'w') as outfile:
  for img in imgs_and_paths:
      caption = caption_prefix + caption_images(img[1]).split("\n")[0]
      entry = {"file_name":img[0].split("/")[-1], "prompt": caption}
      json.dump(entry, outfile)
      outfile.write('\n')

In [ ]:
import gc

# delete the BLIP pipelines and free up some memory
del blip_processor, blip_model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!accelerate config default

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset

dataset = load_dataset("linoyts/3d_icon")
print(dataset)
print(dataset["train"].features)


In [ ]:
!pip uninstall peft -y

In [ ]:
!pip install peft>=0.15.0 -q

In [ ]:
!accelerate launch train_dreambooth_lora_sdxl.py \
--pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
--pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix" \
--dataset_name="./3d_icon_dataset/" \
--caption_column="prompt" \
--output_dir="3dicon_lora_model" \
--mixed_precision="fp16" \
--instance_prompt="a 3D icon in the style of TOK" \
--resolution=1024 \
--train_batch_size=1 \
--gradient_accumulation_steps=4 \
--gradient_checkpointing \
--learning_rate=1e-4 \
--snr_gamma=5.0 \
--lr_scheduler="constant" \
--lr_warmup_steps=0 \
--use_8bit_adam \
--max_train_steps=500 \
--checkpointing_steps=500 \
--seed="0"


In [ ]:
from diffusers import DiffusionPipeline
#from peft import PeftModel  # optional, for merging
import torch

# Load base SDXL pipeline
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", 
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")

# Load LoRA weights into the pipeline
pipe.load_lora_weights("3dicon_lora_model")

# Run inference
prompt = "a chineese man eating noodles in the style of TOK"
image = pipe(prompt, guidance_scale=7.5, num_inference_steps=25).images[0]


In [ ]:
from IPython.display import display
display(image)

In [ ]:
from huggingface_hub import HfApi, whoami

token = ""  

api = HfApi(token=token)
username = whoami(token=token)["name"]
repo_id = f"{username}/3dicon-lora"

# Create the repo under your own namespace
api.create_repo(repo_id=repo_id, repo_type="model", token=token)

# Save the model
pipe.save_pretrained("3dicon_lora_model")

# Upload it
api.upload_folder(
    folder_path="3dicon_lora_model",
    repo_id=repo_id,
    repo_type="model",
    token=token
)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import whoami
from pathlib import Path

output_dir = "3dicon_lora_model" 
username = whoami(token=Path("/root/.cache/huggingface/"))["name"]
repo_id = f"{username}/{output_dir}"

In [ ]:
pip install transformers==4.37.2 --upgrade --force-reinstall

In [ ]:
!pip install -U huggingface_hub>=0.20.0 diffusers>=0.27.0
!pip install --force-reinstall transformers

In [ ]:
pip install numpy==1.26.4 fsspec==2025.3.0 nltk==3.2.4 scipy==1.13.0

In [ ]:
from IPython.display import display, Markdown

link_to_model = f"https://huggingface.co/{repo_id}"
display(Markdown("### Your model has finished training.\nAccess it here: {}".format(link_to_model)))

In [ ]:
import torch
from diffusers import DiffusionPipeline, AutoencoderKL
from huggingface_hub import whoami

# 1. Load base model with optimized settings
vae = AutoencoderKL.from_pretrained(
    "madebyollin/sdxl-vae-fp16-fix",
    torch_dtype=torch.float16
)

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

# 2. Load your LoRA (assuming you trained with "TOK" placeholder)
repo_id = f"{whoami()['name']}/3dicon_lora_model" 
pipe.load_lora_weights(repo_id)



In [ ]:
# 3. Test with proper prompt construction
prompt = "a 3D icon of TOK smartphone, isometric view, pastel colors, clean background"  # Must include "TOK"
negative_prompt = "blurry, low quality, 2D, photograph, realistic"

# 4. Generate with optimized settings
image = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=30,  
    guidance_scale=7.5,      
    cross_attention_kwargs={"scale": 0.8}  # LoRA strength (0.6-1.0)
).images[0]

image

In [ ]:
import torch
from diffusers import DiffusionPipeline, AutoencoderKL

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
)
pipe.load_lora_weights(repo_id)
_ = pipe.to("cuda")

In [ ]:
prompt = "a 3D icon of TOK smartphone, isometric view, pastel colors, clean background" 

image = pipe(prompt=prompt, num_inference_steps=25).images[0]
image

In [ ]:
import torch
from diffusers import DiffusionPipeline, AutoencoderKL

# 1. Load base SDXL model
vae = AutoencoderKL.from_pretrained(
    "madebyollin/sdxl-vae-fp16-fix",
    torch_dtype=torch.float16
)

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")

# 2. Load your trained LoRA (local version)
pipe.load_lora_weights("3dicon_lora_model")  # Your output directory

# 3. Test generation
prompt = "a 3D icon of TOK smartphone, isometric view, pastel colors"  


image = pipe(
    prompt=prompt,
    num_inference_steps=30,
    guidance_scale=7.5,
    cross_attention_kwargs={"scale": 0.8}  
).images[0]

image  

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch
from IPython.display import display

# 1. Load base SDXL model
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

# 2. Load LoRA weights
pipe.load_lora_weights("sgmarwa/3dicon-lora")

# Generate a test image
prompt = "A colorful 3D icon of a magical potion bottle, in the style of TOK"
image = pipe(prompt).images[0]
display(image)


In [ ]:
image.save("outputcolor.png")

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch
from IPython.display import display

# 1. Load base SDXL model
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

# 2. Load LoRA weights
pipe.load_lora_weights("sgmarwa/3dicon-lora")

# Generate a test image
prompt = "a smiling banana in the style of TOK"
image = pipe(prompt).images[0]
display(image)


In [ ]:
image.save("img_banana.png")

In [ ]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

inputs = processor(text=["A 3D icon of a robot cat"], images=Image.open("output_image.png"), return_tensors="pt", padding=True)
outputs = model(**inputs)
similarity = outputs.logits_per_image.softmax(dim=1)


In [ ]:
print("CLIP Score:", similarity.item())

In [ ]:
prompts = ["a robot cat", "icecream cone with sprinkles", "a red pillow with ababy sleeping on it"]
images = ["output_image.png", "img_icecream.png", "img_pillow.png"]
scores = []

for prompt, image_path in zip(prompts, images):
    image = Image.open(image_path)
    inputs = processor(text=[prompt], images=image, return_tensors="pt", padding=True)
    outputs = model(**inputs)
    score = outputs.logits_per_image.softmax(dim=1).item()
    scores.append((prompt, score))

for p, s in scores:
    print(f"{p}: {s:.3f}")

In [ ]:
from torch.nn.functional import cosine_similarity

style_image = Image.open("./3d_icon_dataset/02.jpg")
generated_image = Image.open("output_image.png")

# Get embeddings
style_emb = model.get_image_features(**processor(images=style_image, return_tensors="pt"))
gen_emb = model.get_image_features(**processor(images=generated_image, return_tensors="pt"))

similarity_score = cosine_similarity(style_emb, gen_emb).item()
print (similarity_score)

In [ ]:
from torch.nn.functional import cosine_similarity

style_image = Image.open("./3d_icon_dataset/05.jpg")
generated_image = Image.open("img_icecream.png")

# Get embeddings
style_emb = model.get_image_features(**processor(images=style_image, return_tensors="pt"))
gen_emb = model.get_image_features(**processor(images=generated_image, return_tensors="pt"))

similarity_score = cosine_similarity(style_emb, gen_emb).item()
print (similarity_score)

In [ ]:
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from torch.nn.functional import cosine_similarity
import torch

# Initialize model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Load images
style_image = Image.open("./3d_icon_dataset/22.jpg")
generated_image = Image.open("img_banana.png")

# Get embeddings
style_inputs = processor(images=style_image, return_tensors="pt")
generated_inputs = processor(images=generated_image, return_tensors="pt")

style_emb = model.get_image_features(**style_inputs)
gen_emb = model.get_image_features(**generated_inputs)

# Compute cosine similarity
similarity_score = cosine_similarity(style_emb, gen_emb).item()
print(f"Cosine Similarity Score: {similarity_score:.4f}")


In [4]:
from torchmetrics.image.fid import FrechetInceptionDistance
import torchvision.transforms as T
from PIL import Image
import torch

import os

real_image_dir = "./3d_icon_dataset"
generated_image_dir = "./generated/generated"

real_image_paths = [os.path.join(real_image_dir, f) for f in os.listdir(real_image_dir) if f.endswith(('.png','.jpg','.jpeg'))]
generated_image_paths = [os.path.join(generated_image_dir, f) for f in os.listdir(generated_image_dir) if f.endswith(('.png','.jpg','.jpeg'))]

transform = T.Compose([
    T.Resize((299, 299)),
    T.ToTensor()
])

fid = FrechetInceptionDistance(feature=2048).to("cuda")

for img_path in real_image_paths:
    img = transform(Image.open(img_path)).unsqueeze(0).to("cuda")
    img = (img * 255).to(torch.uint8)
    fid.update(img, real=True)

for img_path in generated_image_paths:
    img = transform(Image.open(img_path)).unsqueeze(0).to("cuda")
    img = (img * 255).to(torch.uint8)
    fid.update(img, real=False)

score = fid.compute()
print("FID Score:", score.item())



FID Score: 322.27508544921875


In [ ]:
!pip install torch-fidelity

In [5]:
print(f"Real images count: {len(real_image_paths)}")
print(f"Generated images count: {len(generated_image_paths)}")

Real images count: 23
Generated images count: 6


In [10]:
import torch
import torchvision.models as models
import torchvision.transforms as T
from PIL import Image

# Load pretrained VGG19
vgg = models.vgg19(pretrained=True).features[:21].eval().cuda()

# Image preprocessing
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])

def gram_matrix(feature_map):
    (b, c, h, w) = feature_map.size()
    features = feature_map.view(b, c, h * w)
    G = torch.bmm(features, features.transpose(1, 2))
    return G / (c * h * w)

def get_features(img, model):
    x = transform(img).unsqueeze(0).cuda()
    for layer in model:
        x = layer(x)
    return x

# Load images
style_img = Image.open("./3d_icon_dataset/20.jpg")
gen_img = Image.open("./generated/generated/img_banana.png")

# Get feature maps
F_style = get_features(style_img, vgg)
F_gen = get_features(gen_img, vgg)

# Compute Gram matrices
G_style = gram_matrix(F_style)
G_gen = gram_matrix(F_gen)

# Compute style similarity (the lower the better)
style_loss = torch.nn.functional.mse_loss(G_style, G_gen)
print("Style Similarity (Gram Loss):", style_loss.item())

Style Similarity (Gram Loss): 3.0336223062477075e-06
